In [2]:
import time
import argparse
import sys
import os
import os.path as osp

import numpy as np
import torch
import pandas as pd
#import pickle5 as pickle
import pickle


import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from sklearn.metrics import r2_score as r2
import matplotlib.pyplot as plt
from itertools import product

import pandas as pd
import os.path as osp
import inspect
from torch_geometric.data import Data
from sklearn import preprocessing


In [4]:
attr=pd.read_csv('attr_clean_new.csv') #bond attributes
net=pd.read_csv('net_clean_new.csv') #bond fund investment data
fund=pd.read_csv('fund_clean_new.csv') #bond fund investment data

attr.drop([attr.columns[0]],axis=1,inplace=True)
net.drop([net.columns[0]],axis=1,inplace=True)
fund.drop([fund.columns[0]],axis=1,inplace=True)

In [5]:
attr

,cusip,month,yq,rat_moodys,rat_low,rat_med,rat_last,ttm,cpn,ai,p,yd,return
0,00184AAC,200207,200203,8.0,8.0,8.0,8.0,344.0,0.000,22.143836,76.0625,10.216908,NaN
1,00184AAC,200208,200203,8.0,8.0,8.0,8.0,343.0,0.000,28.619863,89.5000,8.617000,0.179939
2,00184AAC,200209,200203,8.0,8.0,8.0,8.0,342.0,0.000,35.095890,81.2500,9.549000,-0.082311
3,00184AAC,200210,200204,8.0,8.0,8.0,8.0,341.0,38.125,0.000000,89.7650,8.592000,0.104034
4,00184AAC,200211,200204,8.0,8.0,8.0,8.0,340.0,0.000,9.609589,99.4460,7.671981,0.072982
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,92343VBR,201608,201603,8.0,7.0,7.0,7.0,84.0,0.000,23.704110,117.1790,2.471047,-0.002591
9392,92343VBR,201609,201603,8.0,7.0,7.0,7.0,83.0,25.750,0.000000,116.7250,2.510286,-0.002086
9393,92343VBR,201610,201604,8.0,7.0,7.0,7.0,82.0,0.000,6.349315,114.1720,2.861459,-0.037662
9394,92343VBR,201611,201604,8.0,7.0,7.0,7.0,81.0,0.000,10.723288,110.4240,3.413351,-0.028836


In [6]:
net

,cusip,yq,fundid,paramt
0,00184AAC,200904,14930,2673
1,00184AAC,201001,14930,2673
2,00184AAC,201002,14930,2673
3,00184AAC,201003,14930,2673
4,00184AAC,201004,14930,2130
...,...,...,...,...
51138,92343VBR,201504,87116,282
51139,92343VBR,201601,87116,282
51140,92343VBR,201602,87116,282
51141,92343VBR,201603,87116,446


In [7]:
###make the attr tensor and net tensor


#merge
#1.select common bond and yq in attr and net
#common bond
print('number of bond in bond attributes',len(attr['cusip'].unique()))
print('number of bond in bond fund bipartite graph',len(net['cusip'].unique()))
common_bond=np.intersect1d(attr['cusip'].unique(),net['cusip'].unique())
print(len(common_bond))

#common quarter
print('number of yq in bond attributes',len(attr['yq'].unique()))
print('number of yq in bond fund bipartite graph',len(net['yq'].unique()))
print('number of yq in  fund attr',len(fund['yq'].unique()))
common_yq=np.intersect1d(attr['yq'].unique(),net['yq'].unique())
print(len(common_yq))
common_yq=np.sort(common_yq)

print('number of fund in bond fund bipartite graph',len(net['fundid'].unique()))
print('number of fund in fund attr',len(fund['fundid'].unique()))
common_fund=np.intersect1d(fund['fundid'].unique(),net['fundid'].unique())
print(len(common_fund))
common_fund=np.sort(common_fund)

#2. decide the sequence and net tensor
#3.decide the sequence and net tensor


number of bond in bond attributes 54
number of bond in bond fund bipartite graph 54
54
number of yq in bond attributes 58
number of yq in bond fund bipartite graph 58
number of yq in  fund attr 58
58
number of fund in bond fund bipartite graph 57
number of fund in fund attr 57
57


In [8]:
#define the sequence of bond yq and fund in net data
dfnet=pd.DataFrame(data=product(common_bond,common_fund))
dfnet.columns=['cusip','fundid']

df_net=pd.merge(dfnet, net[net['yq']==common_yq[0]], how='left', on=['cusip','fundid'])

df1=df_net.pivot_table(values='paramt',index='cusip',columns='fundid',aggfunc='sum')

bond_tensor=np.expand_dims(df1.values, axis=2)

In [9]:
for i in range(1,len(common_yq)):
    print(i)
    df_net=pd.merge(dfnet, net[net['yq']==common_yq[i]], how='left', on=['cusip','fundid'])
    print(i,'merge finish')
    df1=df_net.pivot_table(values='paramt',index='cusip',columns='fundid',aggfunc='sum')
    print(i,'pivot finish')
    df2=np.expand_dims(df1.values, axis=2)
    bond_tensor=np.concatenate((bond_tensor,df2),axis=2)
    print(i,bond_tensor.shape)

1
1 merge finish
1 pivot finish
1 (54, 57, 2)
2
2 merge finish
2 pivot finish
2 (54, 57, 3)
3
3 merge finish
3 pivot finish
3 (54, 57, 4)
4
4 merge finish
4 pivot finish
4 (54, 57, 5)
5
5 merge finish
5 pivot finish
5 (54, 57, 6)
6
6 merge finish
6 pivot finish
6 (54, 57, 7)
7
7 merge finish
7 pivot finish
7 (54, 57, 8)
8
8 merge finish
8 pivot finish
8 (54, 57, 9)
9
9 merge finish
9 pivot finish
9 (54, 57, 10)
10
10 merge finish
10 pivot finish
10 (54, 57, 11)
11
11 merge finish
11 pivot finish
11 (54, 57, 12)
12
12 merge finish
12 pivot finish
12 (54, 57, 13)
13
13 merge finish
13 pivot finish
13 (54, 57, 14)
14
14 merge finish
14 pivot finish
14 (54, 57, 15)
15
15 merge finish
15 pivot finish
15 (54, 57, 16)
16
16 merge finish
16 pivot finish
16 (54, 57, 17)
17
17 merge finish
17 pivot finish
17 (54, 57, 18)
18
18 merge finish
18 pivot finish
18 (54, 57, 19)
19
19 merge finish
19 pivot finish
19 (54, 57, 20)
20
20 merge finish
20 pivot finish
20 (54, 57, 21)
21
21 merge finish
21 pi

In [10]:
bond_id=df_net['cusip'].unique()
fund_id=df_net['fundid'].unique()
yq_id=df_net['yq'].unique()
print('finish')
dic_net={'net':bond_tensor,
     'bond_id':bond_id,
     'fund_id':fund_id,
     'yq_id':yq_id
}

finish


In [11]:
import pickle

with open('net_full.pickle', 'wb') as handle:
    pickle.dump(dic_net, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [12]:
#recalculate attr and attr tensor

In [13]:
bond_id1=df_net['cusip'].unique()
month_id1=attr['month'].unique()
yq_id1=attr['yq'].unique()
var_id=['rat_moodys', 'rat_low', 'rat_med', 'rat_last',
       'ttm', 'cpn','ai','p', 'yd', 'return']

In [14]:
attr_tensor=np.expand_dims(attr.loc[attr['month']==month_id1[0],var_id].values , axis=2)
for i in range(1,len(month_id1)):
    df1=attr.loc[attr['month']==month_id1[i],var_id]
    df2=np.expand_dims(df1.values, axis=2)
    attr_tensor=np.concatenate((attr_tensor,df2),axis=2)
    print(i,attr_tensor.shape)

1 (54, 10, 2)
2 (54, 10, 3)
3 (54, 10, 4)
4 (54, 10, 5)
5 (54, 10, 6)
6 (54, 10, 7)
7 (54, 10, 8)
8 (54, 10, 9)
9 (54, 10, 10)
10 (54, 10, 11)
11 (54, 10, 12)
12 (54, 10, 13)
13 (54, 10, 14)
14 (54, 10, 15)
15 (54, 10, 16)
16 (54, 10, 17)
17 (54, 10, 18)
18 (54, 10, 19)
19 (54, 10, 20)
20 (54, 10, 21)
21 (54, 10, 22)
22 (54, 10, 23)
23 (54, 10, 24)
24 (54, 10, 25)
25 (54, 10, 26)
26 (54, 10, 27)
27 (54, 10, 28)
28 (54, 10, 29)
29 (54, 10, 30)
30 (54, 10, 31)
31 (54, 10, 32)
32 (54, 10, 33)
33 (54, 10, 34)
34 (54, 10, 35)
35 (54, 10, 36)
36 (54, 10, 37)
37 (54, 10, 38)
38 (54, 10, 39)
39 (54, 10, 40)
40 (54, 10, 41)
41 (54, 10, 42)
42 (54, 10, 43)
43 (54, 10, 44)
44 (54, 10, 45)
45 (54, 10, 46)
46 (54, 10, 47)
47 (54, 10, 48)
48 (54, 10, 49)
49 (54, 10, 50)
50 (54, 10, 51)
51 (54, 10, 52)
52 (54, 10, 53)
53 (54, 10, 54)
54 (54, 10, 55)
55 (54, 10, 56)
56 (54, 10, 57)
57 (54, 10, 58)
58 (54, 10, 59)
59 (54, 10, 60)
60 (54, 10, 61)
61 (54, 10, 62)
62 (54, 10, 63)
63 (54, 10, 64)
64 (54, 1

In [15]:
attr_tensor1=attr_tensor.copy()
#p -3
#ai -4
#cpn -5

In [16]:
for i in range(attr_tensor1.shape[2]-1):
    attr_tensor1[:,-1,i+1]=[(attr_tensor1[:,-3,i+1]*10+attr_tensor1[:,-4,i+1]+attr_tensor1[:,-5,i+1])-(attr_tensor1[:,-3,i]*10+attr_tensor1[:,-4,i]+attr_tensor1[:,-5,i])]/(attr_tensor1[:,-3,i]*10+attr_tensor1[:,-4,i]+attr_tensor1[:,-5,i])

In [17]:
dic_attr={'attr':attr_tensor1,
     'bond_id':bond_id1,
     'month_id':month_id1,
     'yq_id':yq_id1,
     'var_id':var_id
}

In [18]:
#dummy the rating in attr and impute the ttm to 0

#attr: impute the ttm missing by 0
attr_tensor1[:,4:7,:][np.isnan(attr_tensor1[:,4:7,:])]=0


In [19]:
#attr: make the categorical variables of 4 rating to dummy variable
l=[]
for i in range(4): 
    s = pd.Series(list(attr_tensor1[:,i,:].reshape(-1))) #the i-th rating
    d=pd.get_dummies(s).values.reshape(attr_tensor1.shape[0],attr_tensor1.shape[2],-1)
    d1=np.moveaxis(d,2,1)
    l.append(d1)

dummy_rating=np.concatenate(l,axis=1)
dummy_attr=np.concatenate((dummy_rating,attr_tensor1[:,4:,:]),axis=1)
dic_attr['dummy_attr']=dummy_attr

In [20]:
import pickle

with open('attr_full_new.pickle', 'wb') as handle:
    pickle.dump(dic_attr, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [21]:
fund_id=df_net['fundid'].unique()
yq_id=common_yq
var_id=['holdingbond', 'totalpar', 'matchp',
       'meanpar', 'maxpar', 'minpar', 'stdpar']

In [22]:
fund_tensor=np.expand_dims(fund.loc[fund['yq']==common_yq[0],var_id].values , axis=2)
for i in range(1,len(yq_id)):
    df1=fund.loc[fund['yq']==yq_id[i],var_id]
    df2=np.expand_dims(df1.values, axis=2)
    fund_tensor=np.concatenate((fund_tensor,df2),axis=2)
    print(i,fund_tensor.shape)

1 (57, 7, 2)
2 (57, 7, 3)
3 (57, 7, 4)
4 (57, 7, 5)
5 (57, 7, 6)
6 (57, 7, 7)
7 (57, 7, 8)
8 (57, 7, 9)
9 (57, 7, 10)
10 (57, 7, 11)
11 (57, 7, 12)
12 (57, 7, 13)
13 (57, 7, 14)
14 (57, 7, 15)
15 (57, 7, 16)
16 (57, 7, 17)
17 (57, 7, 18)
18 (57, 7, 19)
19 (57, 7, 20)
20 (57, 7, 21)
21 (57, 7, 22)
22 (57, 7, 23)
23 (57, 7, 24)
24 (57, 7, 25)
25 (57, 7, 26)
26 (57, 7, 27)
27 (57, 7, 28)
28 (57, 7, 29)
29 (57, 7, 30)
30 (57, 7, 31)
31 (57, 7, 32)
32 (57, 7, 33)
33 (57, 7, 34)
34 (57, 7, 35)
35 (57, 7, 36)
36 (57, 7, 37)
37 (57, 7, 38)
38 (57, 7, 39)
39 (57, 7, 40)
40 (57, 7, 41)
41 (57, 7, 42)
42 (57, 7, 43)
43 (57, 7, 44)
44 (57, 7, 45)
45 (57, 7, 46)
46 (57, 7, 47)
47 (57, 7, 48)
48 (57, 7, 49)
49 (57, 7, 50)
50 (57, 7, 51)
51 (57, 7, 52)
52 (57, 7, 53)
53 (57, 7, 54)
54 (57, 7, 55)
55 (57, 7, 56)
56 (57, 7, 57)
57 (57, 7, 58)


In [23]:
#impute nan by 0
fund_tensor[np.isnan(fund_tensor)]=0

In [24]:
dic_fund={'fund':fund_tensor,
     'yq_id':yq_id,
     'var_id':var_id
}

In [25]:
import pickle

with open('fund_full_new.pickle', 'wb') as handle:
    pickle.dump(dic_fund, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [26]:
##########make the data list

#load the data
with open('net_full.pickle', 'rb') as handle:
    dic_net = pickle.load(handle)
net=dic_net['net']#[0:1000,0:1000,:]

with open('attr_full_new.pickle', 'rb') as handle:
    dic_attr = pickle.load(handle)
dummy_attr=dic_attr['dummy_attr']#[0:1000,0:1000,:]


with open('fund_full_new.pickle', 'rb') as handle:
    dic_fund = pickle.load(handle)
fund=dic_fund['fund']#[0:1000,0:1000,:]

In [27]:
net=dic_net['net']
dummy_attr=dic_attr['dummy_attr']
fund=dic_fund['fund']

In [28]:
print(net.shape)
print(dummy_attr.shape)
#print(fund.shape)

(54, 57, 58)
(54, 104, 174)


In [30]:
import pandas as pd
import os.path as osp
import inspect
from torch_geometric.data import Data
from sklearn import preprocessing

import torch
import random
import numpy as np
import pdb


def create_node(df_feature):
    bond_node=df_feature.tolist()
    fund_node=1
    return (bond_node,fund_node)

def create_edge(df):
    #df=df.values
    fund_list=[i+df.shape[0] for i in list(np.where(df!=0)[1])]
    edge_start_new=list(np.where(df!=0)[0])+ fund_list
    edge_end_new=fund_list +list(np.where(df!=0)[0])

    return (edge_start_new, edge_end_new)

def create_edge_attr(df):
    #df=df.values
    edge_attr=list(df[df!=0])+list(df[df!=0])
    return edge_attr

def get_data(df_X, df_yield,df_price,df_return, df_feature,normalize=True,normalize_y=False):
    #df_y = df_y.to_numpy()

    if normalize:
        min_max_scaler = preprocessing.MinMaxScaler()
        df_X = min_max_scaler.fit_transform(df_X)
       
    if normalize_y:
        min_max_scaler_y = preprocessing.MinMaxScaler()
        df_y_norm = min_max_scaler_y.fit_transform(df_y)
        #min_max_scaler_y.inverse_transform(scaled_data)


    edge_start, edge_end = create_edge(df_X)
    edge_index = torch.tensor([edge_start, edge_end], dtype=int)
    edge_attr = torch.tensor(create_edge_attr(df_X), dtype=torch.float).reshape(-1,1)#add a reshape
    bond_node_init,fund_node_init = create_node(df_feature) 
    bond_x = torch.tensor(bond_node_init, dtype=torch.float)
    fund_x = torch.tensor(fund_node_init, dtype=torch.float)
    #y = torch.tensor(df_y_norm, dtype=torch.float)
    df_yield = torch.tensor(df_yield, dtype=torch.float)
    df_price = torch.tensor(df_price, dtype=torch.float)
    df_return = torch.tensor(df_return, dtype=torch.float)
    
   
    data = Data(bond_x=bond_x, fund_x=fund_x,df_yield=df_yield,df_price=df_price,df_return=df_return,edge_index=edge_index, edge_attr=edge_attr,
            )


    
    return data



def load_data1(which_month,dummy_attr,net):
    df_price = dummy_attr[:,-3,which_month] #target
    df_yield = dummy_attr[:,-2,which_month] #target
    df_return = dummy_attr[:,-1,which_month] #target
    df_X = net[:, :,int(which_month//3)] #network
    #df_fund=fund[:,:,int(which_month//3)]
    df_attr=dummy_attr[:,:-3,which_month] #attr
    data = get_data(df_X, df_yield,df_price,df_return,df_attr)
    return data

In [31]:
data_list=[]
for i in range(dummy_attr.shape[2]):
    data=load_data1(i,dummy_attr,net)
    data_list.append(data)
    print(i,'finish')
torch.save(data_list,'data_list.pt')

0 finish
1 finish
2 finish
3 finish
4 finish
5 finish
6 finish
7 finish
8 finish
9 finish
10 finish
11 finish
12 finish
13 finish
14 finish
15 finish
16 finish
17 finish
18 finish
19 finish
20 finish
21 finish
22 finish
23 finish
24 finish
25 finish
26 finish
27 finish
28 finish
29 finish
30 finish
31 finish
32 finish
33 finish
34 finish
35 finish
36 finish
37 finish
38 finish
39 finish
40 finish
41 finish
42 finish
43 finish
44 finish
45 finish
46 finish
47 finish
48 finish
49 finish
50 finish
51 finish
52 finish
53 finish
54 finish
55 finish
56 finish
57 finish
58 finish
59 finish
60 finish
61 finish
62 finish
63 finish
64 finish
65 finish
66 finish
67 finish
68 finish
69 finish
70 finish
71 finish
72 finish
73 finish
74 finish
75 finish
76 finish
77 finish
78 finish
79 finish
80 finish
81 finish
82 finish
83 finish
84 finish
85 finish
86 finish
87 finish
88 finish
89 finish
90 finish
91 finish
92 finish
93 finish
94 finish
95 finish
96 finish
97 finish
98 finish
99 finish
100 finish